In [1]:
import findspark
findspark.init("/h/224/cameron/spark-3.0.0-preview2-bin-hadoop2.7")
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[*]").config("spark.executor.memory", "70g").config("spark.driver.memory", "50g").config(
    "spark.memory.offHeap.enabled", True).config("spark.memory.offHeap.size", "16g").getOrCreate()

In [5]:
# Load the Parquet data
reddit_comments = spark.read.load("/comments_2019.parquet")
reddit_comments.createOrReplaceTempView("comments")
reddit_comments.printSchema()

root
 |-- id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- author: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- body: string (nullable = true)
 |-- edited: integer (nullable = true)
 |-- gilded: string (nullable = true)
 |-- controversiality: integer (nullable = true)
 |-- created_utc: integer (nullable = true)
 |-- distinguished: string (nullable = true)



## Word2Vecf Files
[Word2vecf](https://github.com/BIU-NLP/word2vecf/blob/master/README.md) requires three inputs
* word_vocabulary: file mapping subreddits (strings) to their counts
* count_vocabulary: file mapping users (contexts -> subreddit commenters) to their counts
* training_data: text file of word-context pairs (space delimited)


### Word Vocabulary

In [13]:
word_vocabulary = reddit_comments.groupBy("subreddit").count().cache()
word_vocabulary.show()

+-------------------+-------+
|          subreddit|  count|
+-------------------+-------+
|              anime|3303822|
|       gentlefemdom|  62808|
|           Goldfish|  27526|
|         MLBTheShow| 587781|
|             travel| 386688|
|         costa_rica|  11099|
|       SaltLakeCity| 125313|
|UnresolvedMysteries| 332677|
|     TrueOffMyChest| 726501|
|         traderjoes|  69021|
|             AdPorn|   3857|
|         MensRights| 503714|
|            Amateur| 114799|
|  BeautyGuruChatter| 564832|
|         NHLStreams|  67022|
|          GemsofWar|  11396|
|      gastricsleeve|  31746|
|                NIU|   3486|
|    NewLondonCounty|  22403|
|    Notakeonlythrow|   6424|
+-------------------+-------+
only showing top 20 rows



In [ ]:
count_vocabulary = reddit_comments.groupBy("author").count().cache()
count_vocabulary.show()